In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['HR']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('heart rate', 4111),
 ('homologous recombination', 3668),
 ('hazard ratio', 976),
 ('hypersensitive response', 936),
 ('hormone receptor', 149),
 ('high risk', 130),
 ('hypersensitive reaction', 93),
 ('hypoxia reoxygenation', 81),
 ('histamine release', 59),
 ('homologous recombination repair', 37),
 ('high resolution', 34),
 ('high responders', 34),
 ('hepatic resection', 26),
 ('heptad repeat', 23),
 ('hairy roots', 23),
 ('homologous repair', 21),
 ('heterotrophic respiration', 15),
 ('homologous direct repair', 15),
 ('hairless', 14),
 ('hypersensitive', 12),
 ('halorhodopsin', 12),
 ('hypersensitive cells death response', 12),
 ('hinge region', 11),
 ('high reward', 10),
 ('h + atpase rich', 10),
 ('hypophosphatemic rickets', 9),
 ('human resource', 9),
 ('hypersensitive resistance', 9),
 ('lr and high', 9),
 ('high reduced', 8),
 ('hazard risk', 6),
 ('heal rate', 6),
 ('hwangryunhaedoktang', 6),
 ('hydraulic redistribution', 5),
 ('heteroreceptor', 5),
 ('hit rate', 5),
 ('ho

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'MESH:D006339': 'Heart Rate',
 'GO:GO:0035825': 'homologous recombination',
 'GO:GO:0000725': 'recombinational repair',
 'MESH:D012119': 'Respiration',
 'DOID:DOID:0060056': 'hypersensitivity reaction disease',
 'MESH:D025602': 'Halorhodopsins',
 'MESH:D012201': 'Reward',
 'MESH:D063730': 'Rickets, Hypophosphatemic',
 'MESH:D000078329': 'Workforce',
 'MESH:D012306': 'Risk',
 'MESH:C000626630': 'hwangryunhaedok-tang',
 'GO:GO:0006310': 'DNA recombination',
 'MESH:D011968': 'Receptors, Histamine',
 'EFO:0009785': 'remission',
 'MESH:D012164': 'Retinal Diseases',
 'MESH:D053209': 'Herbicide Resistance',
 'MESH:D006321': 'Heart',
 'MESH:D011995': 'Recombination, Genetic',
 'MESH:D005796': 'Genes',
 'HGNC:5172': 'HR',
 'MESH:D006967': 'Hypersensitivity',
 'MESH:D059765': 'Homologous Recombination'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-10 03:12:00] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'h + atpase rich': 'ungrounded',
  'hairless': 'HGNC:5172',
  'hairless gene': 'HGNC:5172',
  'hairy roots': 'ungrounded',
  'halorhodopsin': 'MESH:D025602',
  'hazard rate': 'ungrounded',
  'hazard ratio': 'ungrounded',
  'hazard risk': 'ungrounded',
  'heal rate': 'ungrounded',
  'heart': 'MESH:D006321',
  'heart rate': 'MESH:D006339',
  'heat rate': 'ungrounded',
  'heifer rebreeding': 'ungrounded',
  'helical region': 'ungrounded',
  'hepatic resection': 'ungrounded',
  'heptad repeat': 'ungrounded',
  'herbicide resistance': 'MESH:D053209',
  'heteroreceptor': 'ungrounded',
  'heterotrophic respiration': 'ungrounded',
  'high novelty responders': 'ungrounded',
  'high reduced': 'ungrounded',
  'high resolution': 'ungrounded',
  'high responders': 'ungrounded',
  'high reward': 'ungrounded',
  'high risk': 'ungrounded',
  'high runner': 'ungrounded',
  'higher risk': 'ungrounded',
  'hindered ratio': 'ungrounded',
  'hinge region': 'ungrounded',
  'histamine receptor': 'FPLX:HRH'

In [11]:
grounding_map, names, pos_labels = [{'h + atpase rich': 'ungrounded',
  'hairless': 'HGNC:5172',
  'hairless gene': 'HGNC:5172',
  'hairy roots': 'ungrounded',
  'halorhodopsin': 'MESH:D025602',
  'hazard rate': 'ungrounded',
  'hazard ratio': 'ungrounded',
  'hazard risk': 'ungrounded',
  'heal rate': 'ungrounded',
  'heart': 'MESH:D006321',
  'heart rate': 'MESH:D006339',
  'heat rate': 'ungrounded',
  'heifer rebreeding': 'ungrounded',
  'helical region': 'ungrounded',
  'hepatic resection': 'ungrounded',
  'heptad repeat': 'ungrounded',
  'herbicide resistance': 'MESH:D053209',
  'heteroreceptor': 'ungrounded',
  'heterotrophic respiration': 'ungrounded',
  'high novelty responders': 'ungrounded',
  'high reduced': 'ungrounded',
  'high resolution': 'ungrounded',
  'high responders': 'ungrounded',
  'high reward': 'ungrounded',
  'high risk': 'ungrounded',
  'high runner': 'ungrounded',
  'higher risk': 'ungrounded',
  'hindered ratio': 'ungrounded',
  'hinge region': 'ungrounded',
  'histamine receptor': 'FPLX:HRH',
  'histamine release': 'GO:GO:0001821',
  'histologic remission': 'ungrounded',
  'histologic response': 'ungrounded',
  'hit rate': 'ungrounded',
  'home range': 'ungrounded',
  'homologous direct repair': 'GO:GO:0035825',
  'homologous dna recombination': 'GO:GO:0035825',
  'homologous dsbs repair': 'GO:GO:0035825',
  'homologous recombination': 'GO:GO:0035825',
  'homologous recombination dna repair': 'GO:GO:0035825',
  'homologous recombination repair': 'GO:GO:0035825',
  'homologous region': 'ungrounded',
  'homologous repair': 'GO:GO:0035825',
  'homologous replacement': 'ungrounded',
  'homologue recombination': 'GO:GO:0035825',
  'hormone receptor': 'ungrounded',
  'hormone replacement': 'ungrounded',
  'human resource': 'ungrounded',
  'human right violations': 'ungrounded',
  'humoral rejection': 'ungrounded',
  'hwangryunhaedok tang': 'MESH:C000626630',
  'hwangryunhaedoktang': 'MESH:C000626630',
  'hybrid resistance': 'ungrounded',
  'hydraulic redistribution': 'ungrounded',
  'hydroxyethyl rutosides': 'ungrounded',
  'hydroxyethylrutosides': 'ungrounded',
  'hyper sensitivity response': 'GO:GO:0009626',
  'hyperpolarizing response': 'ungrounded',
  'hypersensitive': 'GO:GO:0009626',
  'hypersensitive cells death response': 'GO:GO:0009626',
  'hypersensitive defense response': 'GO:GO:0009626',
  'hypersensitive reaction': 'GO:GO:0009626',
  'hypersensitive resistance': 'GO:GO:0009626',
  'hypersensitive resistance response': 'GO:GO:0009626',
  'hypersensitive response': 'GO:GO:0009626',
  'hypertension retinopathy': 'ungrounded',
  'hypophosphatemic rickets': 'MESH:D063730',
  'hypothermic reconditioning': 'ungrounded',
  'hypoxia reoxygenation': 'ungrounded',
  'lr and high': 'ungrounded'},
 {'HGNC:5172': 'HR',
  'MESH:D025602': 'Halorhodopsins',
  'MESH:D006321': 'Heart',
  'MESH:D006339': 'Heart Rate',
  'MESH:D053209': 'Herbicide Resistance',
  'FPLX:HRH': 'HRH',
  'GO:GO:0001821': 'histamine secretion',
  'GO:GO:0035825': 'homologous recombination',
  'MESH:C000626630': 'hwangryunhaedok-tang',
  'GO:GO:0009626': 'plant-type hypersensitive response',
  'MESH:D063730': 'Rickets, Hypophosphatemic'},
 ['GO:GO:0001821', 'GO:GO:0009626', 'GO:GO:0035825', 'HGNC:5172']]



In [12]:
excluded_longforms = []

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 03:35:39] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 03:38:27] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9885466789982431 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'MESH:D006339': 3126,
  'GO:GO:0001821': 55,
  'GO:GO:0009626': 796,
  'ungrounded': 1234,
  'MESH:D025602': 10,
  'GO:GO:0035825': 2704,
  'MESH:D053209': 2,
  'HGNC:5172': 15,
  'MESH:D006321': 3,
  'MESH:D063730': 8,
  'MESH:C000626630': 3,
  'FPLX:HRH': 3},
 'f1': {'mean': 0.988547, 'std': 0.001446},
 'precision': {'mean': 0.988933, 'std': 0.002714},
 'recall': {'mean': 0.988797, 'std': 0.003423},
 'MESH:D053209': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'GO:GO:0001821': {'f1': {'mean': 0.928778, 'std': 0.075262},
  'pr': {'mean': 0.927273, 'std': 0.06803},
  'rc': {'mean': 0.933333, 'std': 0.097183}},
 'ungrounded': {'f1': {'mean': 0.935725, 'std': 0.007978},
  'pr': {'mean': 0.932757, 'std': 0.016663},
  'rc': {'mean': 0.938979, 'std': 0.0097}},
 'GO:GO:0035825': {'f1': {'mean': 0.989108, 'std': 0.002555},
  'pr': {'mean': 0.990385, 'std': 0.002957},
  'rc': {'mean': 0.98785, 'std': 0.004387}

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for HR

Produces the disambiguations:
	HR*	HGNC:5172
	HRH	FPLX:HRH
	Halorhodopsins	MESH:D025602
	Heart	MESH:D006321
	Heart Rate	MESH:D006339
	Herbicide Resistance	MESH:D053209
	Rickets, Hypophosphatemic	MESH:D063730
	histamine secretion*	GO:GO:0001821
	homologous recombination*	GO:GO:0035825
	hwangryunhaedok-tang	MESH:C000626630
	plant-type hypersensitive response*	GO:GO:0009626

Class level metrics:
--------------------
Grounding                         	Count	F1     
                        Heart Rate	3126	0.98248
          homologous recombination*	2704	0.98911
                        Ungrounded	1234	0.93572
plant-type hypersensitive response*	 796	0.99432
               histamine secretion*	  55	0.92878
                                HR*	  15	    0.8
                    Halorhodopsins	  10	0.53333
         Rickets, Hypophosphatemic	   8	    0.8
                             Heart	   3	    0.0
              hwangryunhaedok-tang	   3	    0.0
                     

In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1